# Bad Boys of the National Football League
By Amber Billings and John Ro

#### Data Sources
We initially found an NFL arrests API and found the source from which the API was pulling data, USA Today (https://www.usatoday.com/sports/nfl/arrests/). To connect arrests with any consequences players would have faced after their arrests within the NFL, we pulled a table of NFL suspensions from Wikipedia (https://en.wikipedia.org/wiki/List_of_suspensions_in_the_National_Football_League). The NFL arrests data contained only the team abbreviations and the suspensions table contained only the full team name. We scraped a list of NFL team names and abbreviations from Quizlet (https://quizlet.com/80005100/nfl-teams-by-abbreviation-flash-cards/).

### Import dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from splinter import Browser
from bs4 import BeautifulSoup

### Scraping data on NFL arrests

In [2]:
url = 'https://www.usatoday.com/sports/nfl/arrests/'

In [3]:
tables = pd.read_html(url)
tables

[           Date Team                  Name POS         Case  \
 0    2019-04-26  HOU          Ryan Griffin  TE     Arrested   
 1    2019-04-13  PHI           Jalen Mills  CB     Arrested   
 2    2019-04-11  JAC     Leonard Fournette  RB     Arrested   
 3    2019-04-04  CIN           Mark Walton  RB  Surrendered   
 4    2019-03-01  BAL          Alex Collins  RB     Arrested   
 5    2019-02-15  CIN           Mark Walton  RB     Arrested   
 6    2019-01-26  DET          Trevor Bates  LB     Arrested   
 7    2019-01-26   KC    De''Anthony Thomas  WR     Arrested   
 8    2019-01-23   NO         P.J. Williams  CB     Arrested   
 9    2019-01-16  CIN           Mark Walton  RB      Charged   
 10   2018-12-18  WAS      Montae Nicholson   S     Arrested   
 11   2018-11-24   SF         Reuben Foster  LB     Arrested   
 12   2018-10-30  NYG         Kyle Lauletta  QB     Arrested   
 13   2018-10-22  DEN            Chad Kelly  QB     Arrested   
 14   2018-09-16  LAR           Aaron Ne

In [4]:
arrests_df = tables[0]
arrests_df.columns = ['date', 'team', 'name', 'position', 'case', 'category', 'description', 'outcome']
arrests_df.head(10)

,date,team,name,position,case,category,description,outcome
0,2019-04-26,HOU,Ryan Griffin,TE,Arrested,Public intoxication,Accused of punching out window of hotel in Nas...,Resolution undetermined.
1,2019-04-13,PHI,Jalen Mills,CB,Arrested,Disorderly conduct,Accused of getting into an altercation with NB...,Resolution undetermined.
2,2019-04-11,JAC,Leonard Fournette,RB,Arrested,License,"Pulled over for speeding in Jacksonville, accu...",Resolution undetermined.
3,2019-04-04,CIN,Mark Walton,RB,Surrendered,"Drugs, gun","Accused of carrying a concealed weapon, mariju...",Resolution undetermined. Team released him lat...
4,2019-03-01,BAL,Alex Collins,RB,Arrested,"Drugs, gun",Police responded to report of a car crash and ...,Resolution undetermined. Team released him sam...
5,2019-02-15,CIN,Mark Walton,RB,Arrested,Battery,Accused of misdemeanor and getting into a scuf...,Resolution undetermined.
6,2019-01-26,DET,Trevor Bates,LB,Arrested,Assault,Accused of punching a police officer after not...,Resolution undetermined.
7,2019-01-26,KC,De''Anthony Thomas,WR,Arrested,Drugs,Suspected of possessing marijuana and drug par...,Resolution undetermined.
8,2019-01-23,NO,P.J. Williams,CB,Arrested,DUI,"Pulled over for speeding, accused of driving d...",Resolution undetermined.
9,2019-01-16,CIN,Mark Walton,RB,Charged,Drugs,"Pulled over for not wearing a seat belt, accus...",Resolution undetermined.


### Cleaning arrests data

To transform this data, we first looked through the list to find any errors in names and fixed those. We renamed the column 'case' because that is a reserved word in MySQL. We also sorted by date in ascending order.

In [5]:
arrests_df.loc[arrests_df['name'] == "De''Anthony Thomas", 'name'] = "De'Anthony Thomas"

In [6]:
arrests_df = arrests_df.sort_values(by='date')
arrests_df = arrests_df.reset_index(drop=True)

In [7]:
arrests_df = arrests_df.rename(columns={'case': 'action', 'team': 'abbreviation'})
arrests_df.head(10)

,date,abbreviation,name,position,action,category,description,outcome
0,2000-01-24,DEN,Rod Smith,WR,Arrested,Domestic violence,"Accused of choking, beating and shoving his co...",Pleaded guilty to misdemeanor count of verbal ...
1,2000-01-31,BAL,Ray Lewis,LB,Charged,Murder,Accused of murder in the stabbing deaths of tw...,"Pleaded guilty to obstruction of justice, one ..."
2,2000-02-03,CAR,Fred Lane,RB,Arrested,"Drugs, gun","Police pulled him over in Jackson, Tenn., for ...",Indicted but shot and killed by his wife in Ju...
3,2000-02-05,CIN,Steve Foley,LB,Arrested,Domestic violence,Accused of kicking in the door of his son's mo...,Charge dropped after woman declined to cooperate.
4,2000-02-06,NYJ,Wayne Chrebet,WR,Arrested,Disorderly conduct,Accused of disorderly conduct in altercation o...,"Pleaded guilty to nuisance violation, fined $230."
5,2000-02-06,IND,Keith Elias,RB,Arrested,Disorderly conduct,Accused of resisting arrest and disorderly con...,"Pleaded guilty to nuisance violation, fined $230."
6,2000-02-16,KC,Andre Rison,WR,Arrested,Theft,"Accused of renting but not returning a $1,100 ...","Pleaded no contest, sentenced to one year of p..."
7,2000-02-19,TEN,Benji Olson,OG,Arrested,DUI,Pulled over at a shopping center in Hopkinsvil...,Pleaded to a reduced charge of failing to oper...
8,2000-02-21,NO,Ricky Williams,RB,Arrested,Failure to appear,"Pulled over in Austin, Texas, accused of impro...","Guilty, pleaded $429 in fines."
9,2000-02-22,KC,Tamarick Vanover,WR,Charged,Theft,Accused of assisting in the sale of a stolen v...,"Pleaded guilty, $10,000 fine, $6,241 to the in..."


### Scraping NFL team abbreviations

In [8]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
url = "https://quizlet.com/80005100/nfl-teams-by-abbreviation-flash-cards/"
browser.visit(url)

In [10]:
abbr_list = []
for i in range(32):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    team = soup.find_all(class_='SetPageTerm-wordText')[i]
    nfl_team = team.find('span').text
    
    abbreviation = soup.find_all(class_='SetPageTerm-definitionText')[i]
    nfl_abbr = abbreviation.find('span').text
    
    abbr_list.append({"team": nfl_team, 
                      "abbreviation": nfl_abbr})

In [11]:
abbr_list

[{'team': 'New York Jets', 'abbreviation': 'NYJ'},
 {'team': 'New York Giants', 'abbreviation': 'NYG'},
 {'team': 'Arizona Cardinals', 'abbreviation': 'ARI'},
 {'team': 'Baltimore Ravens', 'abbreviation': 'BAL'},
 {'team': 'Seattle Seahawks', 'abbreviation': 'SEA'},
 {'team': 'Pittsburgh Steelers', 'abbreviation': 'PIT'},
 {'team': 'New England Patriots', 'abbreviation': 'NE'},
 {'team': 'St. Louis Rams', 'abbreviation': 'STL'},
 {'team': 'Oakland Raiders', 'abbreviation': 'OAK'},
 {'team': 'Houston Texans', 'abbreviation': 'HOU'},
 {'team': 'Dallas Cowboys', 'abbreviation': 'DAL'},
 {'team': 'Jacksonville Jaguars', 'abbreviation': 'JAX'},
 {'team': 'San Diego Chargers', 'abbreviation': 'SD'},
 {'team': 'Denver Broncos', 'abbreviation': 'DEN'},
 {'team': 'Indianapolis Colts', 'abbreviation': 'IND'},
 {'team': 'Tennessee Titans', 'abbreviation': 'TEN'},
 {'team': 'Tampa Bay Buccaneers', 'abbreviation': 'TB'},
 {'team': 'Carolina Panthers', 'abbreviation': 'CAR'},
 {'team': 'Buffalo Bill

In [12]:
nfl_abbrevs = pd.DataFrame.from_dict(abbr_list)
nfl_abbrevs.head()

,abbreviation,team
0,NYJ,New York Jets
1,NYG,New York Giants
2,ARI,Arizona Cardinals
3,BAL,Baltimore Ravens
4,SEA,Seattle Seahawks


### Merging arrests and abbreviations tables

We merged the arrests table with the team abbreviations so the data would match what is in the suspensions table. Then we reordered the columns to be consistent with the original arrests table.

In [13]:
arrests_join = pd.merge(arrests_df, nfl_abbrevs, how='outer', on='abbreviation')
arrests_join.head()

,date,abbreviation,name,position,action,category,description,outcome,team
0,2000-01-24,DEN,Rod Smith,WR,Arrested,Domestic violence,"Accused of choking, beating and shoving his co...",Pleaded guilty to misdemeanor count of verbal ...,Denver Broncos
1,2000-08-09,DEN,Bill Romanowski,LB,Indicted,Drugs,Accused of illegaly obtaining prescription die...,Acquitted by jury.,Denver Broncos
2,2000-10-28,DEN,Brian Griese,QB,Arrested,DUI,"Pulled over for speeding near Denver, accused ...","Pleaded guilty, one year of probation.",Denver Broncos
3,2001-03-02,DEN,Dwayne Carswell,TE,Arrested,Domestic violence,Accused of pulling a woman's hair and bruising...,"Diversion program, community service.",Denver Broncos
4,2001-09-02,DEN,Eddie Kennison,WR,Arrested,Disorderly conduct,Accused of inciting a riot at Louisiana nightc...,Charges dropped.,Denver Broncos


In [14]:
nfl_arrests = arrests_join.drop(columns=['abbreviation'])
nfl_arrests = nfl_arrests[['date', 'name', 'position', 'team', 'action', 'category', 'description', 'outcome']]
nfl_arrests.head()

,date,name,position,team,action,category,description,outcome
0,2000-01-24,Rod Smith,WR,Denver Broncos,Arrested,Domestic violence,"Accused of choking, beating and shoving his co...",Pleaded guilty to misdemeanor count of verbal ...
1,2000-08-09,Bill Romanowski,LB,Denver Broncos,Indicted,Drugs,Accused of illegaly obtaining prescription die...,Acquitted by jury.
2,2000-10-28,Brian Griese,QB,Denver Broncos,Arrested,DUI,"Pulled over for speeding near Denver, accused ...","Pleaded guilty, one year of probation."
3,2001-03-02,Dwayne Carswell,TE,Denver Broncos,Arrested,Domestic violence,Accused of pulling a woman's hair and bruising...,"Diversion program, community service."
4,2001-09-02,Eddie Kennison,WR,Denver Broncos,Arrested,Disorderly conduct,Accused of inciting a riot at Louisiana nightc...,Charges dropped.


### Uploading CSV file of NFL suspensions
Pulled from Wikipedia, preliminary transformation in Google Sheets.

To transform this data, we renamed the columns and ordered by date in ascending order.

In [15]:
csv_file = "resources/suspensions.csv"
nfl_bad_boys_df = pd.read_csv(csv_file)
nfl_bad_boys_df.head()

,Date suspended,Suspension length,Name,Position,Team at the time of suspension,Reason
0,2019-03-01,Indefinite,David Irving,DL,Dallas Cowboys,Violating the substance policies
1,2019-02-26,Indefinite,Randy Gregory,DE,Dallas Cowboys,Violating the substance policies
2,2018-12-20,Indefinite,Josh Gordon,WR,New England Patriots,Violating the substance policies
3,2018-12-14,Indefinite,Martavis Bryant,WR,Oakland Raiders,Violating the substance policies
4,2018-12-07,4 games,Darron Lee,LB,New York Jets,Violating the substance policies


In [16]:
nfl_bad_boys_df = nfl_bad_boys_df.rename(columns={'Date suspended': 'date', 'Suspension length': 'length',
                                                  'Name': 'name', 'Position': 'position', 
                                                  'Team at the time of suspension': 'team', 'Reason': 'description'})
nfl_bad_boys_df = nfl_bad_boys_df.sort_values(by='date')
nfl_bad_boys_df = nfl_bad_boys_df.reset_index(drop=True)
nfl_bad_boys_df.head()

,date,length,name,position,team,description
0,1925-12-17,Life (reinstated in 1926),Art Folz,QB,Chicago Cardinals,Involvement in the 1925 Chicago Cardinals–Milw...
1,1947-04-03,Life,Merle Hapes,RB,New York Giants,Fixing the 1946 NFL Championship Game
2,1947-04-03,Life,Frank Filchock,QB,New York Giants,Fixing the 1946 NFL Championship Game
3,1947-11-22,1 game,Jack Matheson,OE,Chicago Bears,Conduct detrimental to the league
4,1963-04-17,Entire 1963 season,Alex Karras,DT,Detroit Lions,Betting on NFL games


### Relational Database

We chose SQL because there are strong consistency in columns from both tables like date, name, position and team. Right now, neither table has a primary key, but if this database was built out more in the future, there could be a players table and a teams table with specific IDs that link between tables.

### Create database connection

In [22]:
connection_string = "root:armistice9@127.0.0.1/nfl_bad_boys"
engine = create_engine(f'mysql+pymysql://{connection_string}')

In [23]:
engine.table_names()

['arrests', 'suspensions']

### Loading DataFrames into database

In [19]:
nfl_arrests.to_sql(name='arrests', con=engine, if_exists='append', index=False)

In [20]:
nfl_bad_boys_df.to_sql(name='suspensions', con=engine, if_exists='append', index=False)